# Audiobooks Conversion Rate

### Problem Definition
Given 2 & 1/2 years of data from audiobook app., I created a ML algorithm to predict conversion rate of customers. Thus, advertising dollars spent on customers who are not likely to purchase again can be reduced. This saved money can be spent on advertising to new customers to create value and growth opportunities.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import scale,StandardScaler,MinMaxScaler
%matplotlib inline

In [3]:
#Seperating raw data into inputs and targets
raw_data = np.loadtxt('Audiobook data - raw.csv', delimiter = ',')
unscaled_inputs = raw_data[:,1:-1]
targets = raw_data[:,-1]

#### Balancing the Dataset

In [4]:
#Balancing dataset so model is BALANCED (conversion rate = 50%). 
#Imbalance in the dataset would ruin the algorithm, therefore..
#Using for loop and a counter to track which indices/observations need to be removed (when 0's > 1's)

conv_targ = int(np.sum(targets))
zero_targ_counter = 0
indices_to_remove = []


for i in range(targets.shape[0]):
    if targets[i] == 0:
        zero_targ_counter += 1
        if zero_targ_counter > conv_targ:
            indices_to_remove.append(i)

In [5]:
len(indices_to_remove)

9610

In [6]:
#Balanced inputs/targets
unscaled_inputs_balanced = np.delete(unscaled_inputs, indices_to_remove, axis = 0)
targets_balanced = np.delete(targets, indices_to_remove, axis = 0)

#### Standardizing Inputs

In [7]:
scaled_inputs = scale(unscaled_inputs_balanced)

#### Shuffling the data

In [8]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_balanced[shuffled_indices]

#### Splitting the data into train, val and test

In [9]:
samples_count = shuffled_indices.shape[0]

#Splitting number of observations into train,val,test - (80/10/10) split
train_samples_count = int(0.8*samples_count)
val_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - val_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

val_inputs = shuffled_inputs[train_samples_count:train_samples_count + val_samples_count]
val_targets = shuffled_targets[train_samples_count:train_samples_count + val_samples_count]

test_inputs = shuffled_inputs[train_samples_count + val_samples_count:]
test_targets = shuffled_targets[train_samples_count + val_samples_count:]

#### Save the 3 datasets in .npz format so it is tensorflow ready.

In [10]:
#This makes them into numpy compressed array (npz file)
#Compressed arrays are dictionaries: the key value pairs are made when saving the file type. key = value
#Where the value is an array, this allows you to store multiple arrays in a single compressed array
#We can easily extract certain arrays from compressed array using specific keys
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_val', inputs = val_inputs, targets = val_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

#### Load the datasets in the tf-friendly format

In [11]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float) #we want to make sure we don't lose data so we load inputs as floats
train_targets = npz['targets'].astype(np.int)

npz_val = np.load('Audiobooks_data_val.npz')
val_inputs = npz_val['inputs'].astype(np.float)
val_targets = npz_val['targets'].astype(np.int)

npz_test = np.load('Audiobooks_data_test.npz')
test_inputs = npz_test['inputs'].astype(np.float)
test_targets = npz_test['targets'].astype(np.int)

#### Creating the model

In [12]:
input_layer = 10
hidden_layer = 50
output_layer = 2


model = tf.keras.Sequential(layers = [
    tf.keras.layers.Dense(units = input_layer),   #Input layer
    tf.keras.layers.Dense(units = hidden_layer, activation = 'relu'),  #First hidden layer
    tf.keras.layers.Dense(units = hidden_layer, activation = 'relu'),  #Second hidden layer
    tf.keras.layers.Dense(units = output_layer, activation = 'softmax')#Output Layer    
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [13]:
batch_size = 124
max_epochs = 100

early_stop = tf.keras.callbacks.EarlyStopping(patience = 5)

model.fit(x = train_inputs,
         y = train_targets,
         batch_size = batch_size,
         epochs = max_epochs,
         verbose = 1,
         callbacks = [early_stop],
         validation_data = (val_inputs,val_targets)          
         )

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 [==============================] - 0s 132us/sample - loss: 0.6345 - accuracy: 0.6027 - val_loss: 0.5716 - val_accuracy: 0.7025
Epoch 2/100
3579/3579 [==============================] - 0s 14us/sample - loss: 0.5146 - accuracy: 0.7374 - val_loss: 0.4803 - val_accuracy: 0.7718
Epoch 3/100
3579/3579 [==============================] - 0s 14us/sample - loss: 0.4374 - accuracy: 0.7720 - val_loss: 0.4196 - val_accuracy: 0.8054
Epoch 4/100
3579/3579 [==============================] - 0s 13us/sample - loss: 0.3994 - accuracy: 0.7843 - val_loss: 0.3933 - val_accuracy: 0.8188
Epoch 5/100
3579/3579 [==============================] - 0s 13us/sample - loss: 0.3808 - accuracy: 0.8005 - val_loss: 0.3759 - val_accuracy: 0.8300
Epoch 6/100
3579/3579 [==============================] - 0s 13us/sample - loss: 0.3747 - accuracy: 0.7955 - val_loss: 0.3700 - val_accuracy: 0.8255
Epoch 7/100
3579/3579 [==============================] - 0s 14us

In [14]:
model.predict(test_inputs, batch_size = batch_size)

array([[6.09124184e-01, 3.90875816e-01],
       [6.05175410e-05, 9.99939442e-01],
       [2.65306085e-01, 7.34693885e-01],
       [5.74327540e-04, 9.99425650e-01],
       [1.00000000e+00, 3.14602430e-18],
       [1.23695523e-01, 8.76304507e-01],
       [5.34597397e-01, 4.65402633e-01],
       [3.63756239e-01, 6.36243761e-01],
       [1.16873282e-11, 1.00000000e+00],
       [5.68981953e-02, 9.43101823e-01],
       [6.03532314e-01, 3.96467626e-01],
       [1.00000000e+00, 4.38154330e-24],
       [1.80292711e-01, 8.19707274e-01],
       [2.83114910e-01, 7.16885090e-01],
       [3.43430511e-05, 9.99965668e-01],
       [7.99547095e-10, 1.00000000e+00],
       [6.10822201e-01, 3.89177829e-01],
       [9.94064629e-01, 5.93534112e-03],
       [1.58393402e-02, 9.84160721e-01],
       [5.26712656e-01, 4.73287344e-01],
       [6.41170573e-07, 9.99999404e-01],
       [1.00000000e+00, 3.28365564e-11],
       [7.02868477e-02, 9.29713130e-01],
       [5.05937517e-01, 4.94062454e-01],
       [1.052070

In [17]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 36us/sample - loss: 0.2926 - accuracy: 0.8460


In [18]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.29. Test accuracy: 84.60%


#### Predictions (Implementation of Alg.)

In [22]:
predictions_probabilities = model.predict(test_inputs, batch_size = 64)

def round_prob(probability):
    predictions = []
    for i in probability:
        if i[0] < 0.500000:
            predictions.append(0)
        else:
            predictions.append(1)
    return predictions

predictions = round_prob(predictions_probabilities)